In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neural_network import MLPClassifier
import joblib

In [49]:
df = pd.read_csv('/csvPH.csv' , sep=';', encoding='latin1')

In [50]:
num_cols = [c for c in df.columns if c.lower() != 'index']

In [51]:
# Bersihkan dan konversi koma ke titik, lalu numeric
for c in num_cols:
    df[c] = df[c].astype(str).str.replace(r'[^\d,.\-]', '', regex=True)
    df[c] = df[c].str.replace(',', '.')
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Label sesuai aturan
def label_row(row):
    if (6 < row['pH'] < 7) and (20 < row['Temperatur (°C)'] < 30) and (20 < row['Kadar Air (%)'] < 40) and (60 < row['Kelembapan (%)'] < 80):
        return 'ok'
    else:
        return 'warning'

In [52]:
df['label'] = df.apply(label_row, axis=1)

# Fitur & target
X = df[num_cols].values
y = df['label'].values

# Encode label
le = LabelEncoder(); y_enc = le.fit_transform(y)

# Buang baris NaN bila ada
mask = ~np.isnan(X).any(axis=1)
X = X[mask]; y_enc = y_enc[mask]
df = df.iloc[mask]

# Simpan indeks asli
indices = df.index.values

# Split data beserta indeks
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y_enc, indices,
    test_size=0.2,
    random_state=42,
    stratify=y_enc
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ANN (MLP)
mlp = MLPClassifier(
    hidden_layer_sizes=(32,16),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42,
    early_stopping=True,
    n_iter_no_change=15
)
mlp.fit(X_train_scaled, y_train)

# Prediksi
y_pred = mlp.predict(X_test_scaled)
y_pred_labels = le.inverse_transform(y_pred)

# Tampilkan jumlah label setelah klasifikasi
print("Jumlah label setelah klasifikasi:")
print(pd.Series(y_pred_labels).value_counts(), "\n")

# Indeks asli untuk prediksi 'ok' dan 'warning'
ok_indices = idx_test[y_pred_labels == 'ok']
warning_indices = idx_test[y_pred_labels == 'warning']

print("Indeks asli prediksi 'ok'     :", ok_indices.tolist())
print("Indeks asli prediksi 'warning':", warning_indices.tolist(), "\n")

# Evaluasi model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Jumlah label setelah klasifikasi:
warning    1000
Name: count, dtype: int64 

Indeks asli prediksi 'ok'     : []
Indeks asli prediksi 'warning': [360, 1153, 1626, 2070, 4680, 304, 3282, 4517, 422, 1285, 1360, 3920, 67, 2706, 2708, 4537, 2214, 818, 2088, 4837, 4650, 252, 1917, 2343, 727, 3732, 3294, 819, 2523, 514, 2312, 1693, 4674, 3073, 4929, 1026, 1671, 2479, 4430, 1984, 2224, 774, 667, 3728, 3493, 4363, 4574, 4386, 3015, 2405, 3162, 1339, 1260, 992, 2074, 4160, 3067, 961, 3288, 2409, 1086, 4442, 3250, 2461, 4963, 3910, 4396, 649, 2992, 1978, 1253, 3696, 1313, 1915, 1171, 4230, 855, 288, 4054, 2296, 2301, 4207, 4035, 2853, 1824, 2788, 3507, 3009, 675, 4972, 4868, 3602, 2973, 2234, 78, 4012, 3103, 128, 1836, 4521, 4762, 4583, 1141, 3797, 2781, 3267, 1612, 620, 3088, 2934, 4648, 1196, 3618, 959, 1219, 2606, 1979, 976, 148, 4661, 2050, 1307, 903, 432, 4588, 2406, 2371, 766, 1146, 2948, 3136, 9, 2397, 2035, 3252, 2265, 898, 4304, 1723, 3655, 2658, 4873, 2576, 4803, 3664, 3299, 1953, 4368

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv('/csvPH.csv', sep=';', encoding='latin1')

# Pilih kolom numerik (kecuali Index)
num_cols = [c for c in df.columns if c.lower() != 'index']

# Bersihkan data numerik
for c in num_cols:
    df[c] = df[c].astype(str).str.replace(r'[^\d,.\-]', '', regex=True)
    df[c] = df[c].str.replace(',', '.')
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Buat label
def label_row(row):
    if (6 < row['pH'] < 7) and (20 < row['Temperatur (°C)'] < 30) and (20 < row['Kadar Air (%)'] < 40) and (60 < row['Kelembapan (%)'] < 80):
        return 'ok'
    else:
        return 'warning'

df['label'] = df.apply(label_row, axis=1)

# Distribusi label sebelum klasifikasi
print("Distribusi label awal:")
print(df['label'].value_counts(), "\n")

# Fitur & target
X = df[num_cols].values
y = df['label'].values

# Encode label
le = LabelEncoder()
y_enc = le.fit_transform(y)

# Buang baris NaN
mask = ~np.isnan(X).any(axis=1)
X = X[mask]
y_enc = y_enc[mask]
df = df.iloc[mask]

# Simpan indeks asli
indices = df.index.values

# Split data + indeks
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y_enc, indices,
    test_size=0.2,
    random_state=42,
    stratify=y_enc
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- SMOTE Oversampling ---
print("Distribusi sebelum SMOTE:", np.bincount(y_train))
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)
print("Distribusi sesudah SMOTE:", np.bincount(y_train_res), "\n")

# ANN
mlp = MLPClassifier(
    hidden_layer_sizes=(32,16),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42,
    early_stopping=True,
    n_iter_no_change=15
)
mlp.fit(X_train_res, y_train_res)

# Prediksi
y_pred = mlp.predict(X_test_scaled)
y_pred_labels = le.inverse_transform(y_pred)

# Jumlah label setelah klasifikasi
print("Distribusi label setelah klasifikasi:")
print(pd.Series(y_pred_labels).value_counts(), "\n")

# Indeks asli untuk prediksi 'ok' dan 'warning'
ok_indices = idx_test[y_pred_labels == 'ok']
warning_indices = idx_test[y_pred_labels == 'warning']

print("Indeks asli prediksi 'ok'     :", ok_indices.tolist())
print("Indeks asli prediksi 'warning':", warning_indices.tolist(), "\n")

# Evaluasi
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Distribusi label awal:
label
warning    4973
ok           27
Name: count, dtype: int64 

Distribusi sebelum SMOTE: [  22 3978]
Distribusi sesudah SMOTE: [3978 3978] 

Distribusi label setelah klasifikasi:
warning    990
ok          10
Name: count, dtype: int64 

Indeks asli prediksi 'ok'     : [1285, 2312, 1694, 2756, 1703, 4499, 2071, 4334, 1704, 2373]
Indeks asli prediksi 'warning': [360, 1153, 1626, 2070, 4680, 304, 3282, 4517, 422, 1360, 3920, 67, 2706, 2708, 4537, 2214, 818, 2088, 4837, 4650, 252, 1917, 2343, 727, 3732, 3294, 819, 2523, 514, 1693, 4674, 3073, 4929, 1026, 1671, 2479, 4430, 1984, 2224, 774, 667, 3728, 3493, 4363, 4574, 4386, 3015, 2405, 3162, 1339, 1260, 992, 2074, 4160, 3067, 961, 3288, 2409, 1086, 4442, 3250, 2461, 4963, 3910, 4396, 649, 2992, 1978, 1253, 3696, 1313, 1915, 1171, 4230, 855, 288, 4054, 2296, 2301, 4207, 4035, 2853, 1824, 2788, 3507, 3009, 675, 4972, 4868, 3602, 2973, 2234, 78, 4012, 3103, 128, 1836, 4521, 4762, 4583, 1141, 3797, 2781, 3267, 1612, 62